### 데이터 준비하기: 조인,병합,변형

#### 계층적 색인

In [7]:
import pandas as pd
import numpy as np

In [9]:
data = pd.Series(np.random.uniform(size=9),
                 index = [["a","a","a","b","b","c","c","d","d"],
                          [1,2,3,1,3,1,2,2,3]])
data

a  1    0.697734
   2    0.638580
   3    0.008238
b  1    0.719856
   3    0.927198
c  1    0.289119
   2    0.799142
d  2    0.051746
   3    0.465024
dtype: float64

In [11]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [13]:
data["b"]

1    0.719856
3    0.927198
dtype: float64

In [15]:
data["b":"c"]

b  1    0.719856
   3    0.927198
c  1    0.289119
   2    0.799142
dtype: float64

In [17]:
data.loc[["b","d"]]

b  1    0.719856
   3    0.927198
d  2    0.051746
   3    0.465024
dtype: float64

In [19]:
data.loc[:,2]

a    0.638580
c    0.799142
d    0.051746
dtype: float64

In [21]:
data.unstack()

,1,2,3
a,0.697734,0.638580,0.008238
b,0.719856,NaN,0.927198
c,0.289119,0.799142,NaN
d,NaN,0.051746,0.465024


In [23]:
data.unstack().stack()

a  1    0.697734
   2    0.638580
   3    0.008238
b  1    0.719856
   3    0.927198
c  1    0.289119
   2    0.799142
d  2    0.051746
   3    0.465024
dtype: float64

In [25]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index = [["a","a","b","b"], [1,2,1,2]],
                     columns = [["Ohio","Ohio","Colorado"],
                                ["Green","Red","Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [27]:
frame.index.names = ["key1","key2"]
frame.columns.names = ["state","color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [29]:
frame.index.nlevels

2

In [31]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [33]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio","Colorado"],
                           ["Green","Red","Green"]],
                          names = ["state","color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

#### 계층의 순서를 바꾸고 정렬기기

In [36]:
frame.swaplevel("key1","key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [38]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [40]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

#### 계층별 요약 통계

In [43]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [45]:
frame.groupby(level="color",axis="columns").sum()

C:\Users\JuhyunN\AppData\Local\Temp\ipykernel_8900\3295922660.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color",axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

#### DataFrame의 열 사용기기

In [48]:
frame= pd.DataFrame({"a":range(7), "b":range(7,0,-1), "c":["one","one","one","two","two","two","two"], 
                     "d": [0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [50]:
frame2 = frame.set_index(["c","d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [52]:
frame.set_index(["c","d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [54]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


#### 데이터 합치기

##### 데이터베이스 스타일로 DataFrame 합치기

In [63]:
df1 = pd.DataFrame({"key": ["b","b","a","c","a","a","b"],
                    "data1": pd.Series(range(7), dtype= "Int64")})
df2 = pd.DataFrame({"key": ["a","b","d"],
                    "data2": pd.Series(range(3), dtype = "Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [65]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [67]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [69]:
pd.merge(df1,df2, on = "key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [73]:
df3 = pd.DataFrame({"lkey": ["b","b","a","c","a","a","b"],
                    "data1": pd.Series(range(7), dtype= "Int64")})
df4 = pd.DataFrame({"rkey": ["a","b","d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
pd.merge(df3, df4, left_on = "lkey", right_on = "rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [75]:
pd.merge(df1,df2,how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [77]:
pd.merge(df3,df4,left_on = "lkey",right_on = "rkey", how= "outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [79]:
df1 = pd.DataFrame({"key" : ["b","b","a","c","a","b"],
                    "data1": pd.Series(range(6), dtype= 'Int64')})
df2= pd.DataFrame({"key": ["a","b","a","b","d"],
                   "data2": pd.Series(range(5),dtype = 'Int64')})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [81]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [83]:
pd.merge(df1,df2, on="key",how = "left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [85]:
pd.merge(df1,df2, how = "inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [87]:
left = pd.DataFrame({"key1":["foo","foo","bar"],
                     "key2":["one","two","one"],
                     "lval": pd.Series([1,2,3], dtype='Int64')})
right = pd.DataFrame({"key1": ["foo","foo","bar","bar"],
                      "key2": ["one","one","one","two"],
                      "rval": pd.Series([4,5,6,7], dtype = 'Int64')})
pd.merge(left, right, on = ["key1","key2"], how= "outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [93]:
pd.merge(left, right, on = "key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [95]:
pd.merge(left, right, on = "key1", suffixes = ("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


#### 색인 병합하기

In [99]:
left1 = pd.DataFrame({"key": ["a","b","a","a","b","c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1 = pd.DataFrame({"group_val": [3.5,7]}, index=["a","b"])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [101]:
right1

,group_val
a,3.5
b,7.0


In [103]:
pd.merge(left1, right1, left_on = "key", right_index = True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [105]:
pd.merge(left1, right1, left_on ="key", right_index = True, how = "outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [109]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada"],
                      "key2": [2000,2001,2002,2001,2002],
                      "data": pd.Series(range(5),dtype="Int64")})

righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada","Nevada","Ohio","Ohio","Ohio","Ohio"],
        [2001,2000,2000,2000,2001,2002]
    ]
)
righth = pd.DataFrame({"event1": pd.Series([0,2,4,6,8,10], dtype = "Int64",
                                           index = righth_index),
                       "event2": pd.Series([1,3,5,7,9,11], dtype = "Int64",
                                           index = righth_index)})

lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [111]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [113]:
pd.merge(lefth, righth, left_on = ["key1", "key2"], right_index = True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [115]:
pd.merge(lefth, righth, left_on = ["key1", "key2"], 
         right_index = True, how= "outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [117]:
left2 = pd.DataFrame([[1.,2.], [3.,4.], [5.,6.]],
                     index = ["a", "c","e"],
                     columns = ["Ohio", "Nevada"]).astype("Int64")
right2 = pd.DataFrame([[7.,8.], [9.,10.], [11.,12.], [13, 14]],
                      index =["b","c","d","e"],
                      columns = ["Missouri", "Alabama"]).astype("Int64")
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [119]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [123]:
pd.merge(left2, right2, how= "outer", left_index = True, right_index= True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [125]:
left2.join(right2, how ="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


In [127]:
left1.join(right1, on = "key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [131]:
another = pd.DataFrame([[7.,8.],[9.,10.], [11.,12.],[16.,17.]],
                        index = ["a", "c","e","f"],
                        columns = ["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [133]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [135]:
left2.join([right2, another], how = "outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


#### 축 따라 이어 붙이기

In [140]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [142]:
np.concatenate([arr,arr], axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [146]:
s1 = pd.Series([0,1], index = ["a","b"], dtype = "Int64")
s2 = pd.Series([2,3,4], index = ["c","d","e"], dtype = "Int64")
s3 = pd.Series([5,6], index = ["f","g"], dtype = "Int64")
s1

a    0
b    1
dtype: Int64

In [148]:
s2

c    2
d    3
e    4
dtype: Int64

In [150]:
s3

f    5
g    6
dtype: Int64

In [152]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [154]:
pd.concat([s1, s2,s3], axis = "columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [156]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [158]:
pd.concat([s1,s4], axis = "columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [160]:
pd.concat([s1,s4], axis = "columns", join = "inner")

,0,1
a,0,0
b,1,1


In [166]:
result = pd.concat([s1,s1,s3], keys = ["own", "two", "three"])
result

own    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [168]:
result.unstack()

,a,b,f,g
own,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [170]:
pd.concat([s1,s2,s3], axis = "columns", keys = ["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [172]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ["a","b","c"],
                   columns = ["one", "two"])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2),index = ["a","c"],
                   columns = ["three", "four"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [174]:
df2

,three,four
a,5,6
c,7,8


In [176]:
pd.concat([df1, df2], axis = "columns", keys =["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [178]:
pd.concat({"level1": df1, "level2":df2}, axis = "columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [180]:
pd.concat([df1, df2] , axis = "columns", keys = ["level1", "level2"],
          names = ["upper","lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [182]:
df1 = pd.DataFrame(np.random.standard_normal((3,4)),
                   columns = ["a","b","c","d"])
df2 = pd.DataFrame(np.random.standard_normal((2,3)),
                   columns = ["b","d","a"])
df1

,a,b,c,d
0,1.665737,2.816866,-1.208371,-0.278139
1,0.993995,1.162607,-0.021897,0.197226
2,-1.965134,0.753522,1.720580,0.177330


In [184]:
df2

,b,d,a
0,-0.624972,-0.051777,-0.758246
1,-0.300030,-2.445085,0.494090


In [186]:
pd.concat([df1, df2], ignore_index = True)

,a,b,c,d
0,1.665737,2.816866,-1.208371,-0.278139
1,0.993995,1.162607,-0.021897,0.197226
2,-1.965134,0.753522,1.720580,0.177330
3,-0.758246,-0.624972,NaN,-0.051777
4,0.494090,-0.300030,NaN,-2.445085


#### 겹치는 데이터 합치기

In [190]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index = ["f", "e", "d","c","b","a"])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index = ["a", "b", "c","d","e","f"])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [192]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [194]:
np.where(pd.isna(a), b,a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [196]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [198]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2,18,4)})
df2 = pd.DataFrame({"a": [5.,4., np.nan, 3., 7.], 
                    "b": [np.nan, 3.,4.,6.,8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [200]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [202]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### 재구성과 피벗

In [207]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index = pd.Index(["Ohio", "Colorado"], name = "state"),
                    columns = pd.Index(["one", "two", "three"],
                    name = "number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [209]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [211]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [215]:
result.unstack(level = 0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [217]:
result.unstack(level = "state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [221]:
s1 = pd.Series([0,1,2,3], index = ["a","b","c","d"], dtype = "Int64")
s2 = pd.Series([4,5,6], index = ["c","d","e"], dtype = "Int64")
data2 = pd.concat([s1,s2], keys = ["one","two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [223]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [225]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [227]:
data2.unstack().stack(dropna =False)

C:\Users\JuhyunN\AppData\Local\Temp\ipykernel_8900\3590764459.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna =False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [231]:
df = pd.DataFrame({"left": result , "right": result + 5},
                  columns = pd.Index(["left", "right"], name = "side"))
df                

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [233]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [237]:
df.unstack(level="state").stack(level = "side")

C:\Users\JuhyunN\AppData\Local\Temp\ipykernel_8900\260093379.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level="state").stack(level = "side")


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

#### 긴 형식에서 넓은 형식으로 피벗하기

In [259]:
data = pd.read_csv("macrodata.csv")
data = data.loc[:, ["year","quarter", "realgdp", "infl","unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959.0,1.0,2710.349,0.00,5.8
1,1959.0,2.0,2778.801,2.34,5.1
2,1959.0,3.0,2775.488,2.74,5.3
3,1959.0,4.0,2785.204,0.27,5.6
4,1960.0,1.0,2847.699,2.31,5.2


In [263]:
periods = pd.PeriodIndex(year=data.pop("year"),
                          quarter=data.pop("quarter"),
                          name="date")

periods

C:\Users\JuhyunN\AppData\Local\Temp\ipykernel_8900\3707983646.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [265]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [269]:
data =data.reindex(columns = ["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


In [275]:
long_data = (data.stack()
             .reset_index()
             .rename(columns = {0: "value"}))
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


In [277]:
pivoted = long_data.pivot(index = "date", columns = "item",
                          values = "value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [279]:
long_data.index.name = None
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,-0.362197
1,1959-01-01,infl,0.000,0.001272
2,1959-01-01,unemp,5.800,-0.300192
3,1959-04-01,realgdp,2778.801,-0.225887
4,1959-04-01,infl,2.340,-0.187455
5,1959-04-01,unemp,5.100,-0.135506
6,1959-07-01,realgdp,2775.488,-0.907849
7,1959-07-01,infl,2.740,0.100913
8,1959-07-01,unemp,5.300,-1.025481
9,1959-10-01,realgdp,2785.204,0.559660


In [281]:
pivoted = long_data.pivot(index = "date", columns = "item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.001272 -0.362197 -0.300192
1959-04-01  2.34  2778.801   5.1 -0.187455 -0.225887 -0.135506
1959-07-01  2.74  2775.488   5.3  0.100913 -0.907849 -1.025481
1959-10-01  0.27  2785.204   5.6  0.385420  0.559660 -0.728793
1960-01-01  2.31  2847.699   5.2 -1.582215  0.427999  2.053318

In [283]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


In [285]:
unstacked = long_data.set_index(["date", "item"]).unstack(level = 'item')
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.001272 -0.362197 -0.300192
1959-04-01  2.34  2778.801   5.1 -0.187455 -0.225887 -0.135506
1959-07-01  2.74  2775.488   5.3  0.100913 -0.907849 -1.025481
1959-10-01  0.27  2785.204   5.6  0.385420  0.559660 -0.728793
1960-01-01  2.31  2847.699   5.2 -1.582215  0.427999  2.053318

#### 넓은 형식에서 긴 형식으로 피벗하기

In [289]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1,2,3] ,
                   "B": [4,5,6],
                   "C": [7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [291]:
melted = pd.melt(df,id_vars = "key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [295]:
reshaped = melted.pivot(index = "key", columns = "variable", 
                        values = "value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [297]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [299]:
pd.melt(df, id_vars = "key", value_vars = ["A","B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [301]:
pd.melt(df, value_vars = ["A","B","C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [303]:
pd.melt(df, value_vars = ["key", "A","B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
